Convert File from PDF to CSV. Decertified Officers in the state of NH as of 4/1/23.

In [33]:
import pdfplumber
import pandas as pd

file = 'Certified_Officers_as_of_04012023.pdf'

In [34]:
lines = []

with pdfplumber.open(file) as pdf:
    pages = pdf.pages
    for page in pdf.pages:
        text = page.extract_text(extraction_mode="layout")
        for line in text.split('\n'):
            lines.append(line)

Creates Dataframe, but Layout Extraction mode unable to distinguish whitespace in the source pdf. So, only one column per row.

In [39]:
raw_output_df = pd.DataFrame(lines).drop([0, 1]) # Remove headers.
raw_output_df

,0
2,Abbate Christopher NH DOS State Police
3,Abdulkadir Awess Derry
4,Abdulkadir Noah NH DOC
5,Abel Brandan Woodstock
6,"Abel Michael New Ipswich, Wilton"
...,...
3901,Zilinski Colby Swanzey
3902,Zipf Nathan NH DOS State Police
3903,Zobel Matthew Northwood
3904,Zullo Patrick Springfield


Use Space Delimiter and other funcs to create columns Last/First/Suffix/Dept. Only issue is for those with two names in Last or First column. So for those, I have to manually fix. Hopefully, this is sorted out with better understanding of the PyPDF documentation.

In [41]:
raw_output = []
for i in range(len(raw_output_df.index)):
    raw_output.append(raw_output_df.iloc[i][0].split(' '))

for row in raw_output:
    print(row)

['Abbate', 'Christopher', 'NH', 'DOS', 'State', 'Police']
['Abdulkadir', 'Awess', 'Derry']
['Abdulkadir', 'Noah', 'NH', 'DOC']
['Abel', 'Brandan', 'Woodstock']
['Abel', 'Michael', 'New', 'Ipswich,', 'Wilton']
['Abele', 'Christopher', 'Manchester']
['Abele', 'George', 'Rockingham', 'County', 'SO']
['Abele', 'Kathryn', 'Epping']
['Abruzese', 'Michael', 'Manchester']
['Accardi', 'John', 'Coos', 'County', 'SO']
['Accorto', 'Michael', 'Derry']
['Acres', 'Robert', 'Rockingham', 'County', 'SO']
['Adam', 'Michael', 'Bradford']
['Adams', 'Eric', 'Laconia']
['Adams', 'Stephen', 'Webster']
['Adams', 'William', 'North', 'Hampton']
['Adamson', 'William', 'III', 'Nashua']
['Addonizio', 'Brandon', 'Newton']
['Addonizio', 'Richard', 'Milford']
['Adinolfo', 'Jonathan', 'Warner']
['Agraz', 'Justin', 'Merrimack']
['Ahnen', 'Matthew', 'Hillsborough', 'County', 'SO']
['Aikawa', 'Timon', 'Londonderry']
['Ainsworth', 'Kevin', 'Manchester']
['Akerstrom', 'Robert', 'Gilmanton,', 'Loudon']
['Akerstrom', 'Stephe

In [51]:
suffixes = ['Jr.', 'Sr.', 'II', 'III', 'IV', 'V']
def has_suffix(row):
    return any(x in suffixes for x in row)
def get_ind_suffix(row):
    return [i for i, e in enumerate(row) if e in suffixes][0]

In [54]:
output = []
for row in raw_output:
    if has_suffix(row):
        ind_suffix = get_ind_suffix(row)
        output_line = [row[0], ' '.join(row[1:ind_suffix]), row[ind_suffix], ' '.join(row[ind_suffix+1:])]
    else:
        output_line = [row[0], row[1], None, ' '.join(row[2:])]
    output.append(output_line)
output

[['Abbate', 'Christopher', None, 'NH DOS State Police'],
 ['Abdulkadir', 'Awess', None, 'Derry'],
 ['Abdulkadir', 'Noah', None, 'NH DOC'],
 ['Abel', 'Brandan', None, 'Woodstock'],
 ['Abel', 'Michael', None, 'New Ipswich, Wilton'],
 ['Abele', 'Christopher', None, 'Manchester'],
 ['Abele', 'George', None, 'Rockingham County SO'],
 ['Abele', 'Kathryn', None, 'Epping'],
 ['Abruzese', 'Michael', None, 'Manchester'],
 ['Accardi', 'John', None, 'Coos County SO'],
 ['Accorto', 'Michael', None, 'Derry'],
 ['Acres', 'Robert', None, 'Rockingham County SO'],
 ['Adam', 'Michael', None, 'Bradford'],
 ['Adams', 'Eric', None, 'Laconia'],
 ['Adams', 'Stephen', None, 'Webster'],
 ['Adams', 'William', None, 'North Hampton'],
 ['Adamson', 'William', 'III', 'Nashua'],
 ['Addonizio', 'Brandon', None, 'Newton'],
 ['Addonizio', 'Richard', None, 'Milford'],
 ['Adinolfo', 'Jonathan', None, 'Warner'],
 ['Agraz', 'Justin', None, 'Merrimack'],
 ['Ahnen', 'Matthew', None, 'Hillsborough County SO'],
 ['Aikawa', 'Tim

Compile as Output DataFrame

In [55]:
output_df = pd.DataFrame(output, columns=['Last Name', 'First Name', 'Suffix', 'Organizations/Agencies'])
output_df

,Last Name,First Name,Suffix,Organizations/Agencies
0,Abbate,Christopher,None,NH DOS State Police
1,Abdulkadir,Awess,None,Derry
2,Abdulkadir,Noah,None,NH DOC
3,Abel,Brandan,None,Woodstock
4,Abel,Michael,None,"New Ipswich, Wilton"
...,...,...,...,...
3899,Zilinski,Colby,None,Swanzey
3900,Zipf,Nathan,None,NH DOS State Police
3901,Zobel,Matthew,None,Northwood
3902,Zullo,Patrick,None,Springfield


In [56]:
output_df.to_csv('Certified_Officers_as_of_04012023.csv')
print('done')

done
